In [ ]:
import re
import time
import json
import hashlib
import requests
import numpy as np
import tensorflow as tf
from absl import logging
from notifier import notify
from alive_progress import alive_bar
from keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg19 import VGG19
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import (
  ReduceLROnPlateau,
  EarlyStopping,
  ModelCheckpoint,
  TensorBoard
)
from tensorflow.keras import mixed_precision
# Equivalent to the two lines above
mixed_precision.set_global_policy('mixed_float16')

import os
import pandas as pd
import random as rd
import pickle as pkl
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

SEED=42
DATASET_PATH='./data/animes'
DATASET_JSON_PATH='./data/anime_data.json'
AMOUNT_TABLE_PATH='./data/anime_amount.pkl'
DATASET_JSON_RANK='./data/anime_rank.json'
TFRECORD_PATH='./data/anime_data.tfrecord'
TG_ID="293701727"
TG_TOKEN="1878628343:AAEFVRsqDz63ycmaLOFS7gvsG969wdAsJ0w"
WEBHOOK_URL="https://discord.com/api/webhooks/796406472459288616/PAkiGGwqe0_PwtBxXYQvOzbk78B4RQP6VWRkvpBtw6Av0sc_mDa3saaIlwVPFjOIeIbt"

#seed random seed to 42 for reproducibility
rd.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)

### Dataset anime

In [ ]:
def dowload_image(url, anime_name, idx):
  #download image from url
  file_path = f'./data/animes/{anime_name}____{idx}.jpg' 
  if os.path.exists(file_path):
    return

  img_data = requests.get(url).content
  with open(file_path, 'wb') as handler:
    handler.write(img_data)

@notify(
  chat_id=TG_ID,
  api_token=TG_TOKEN,
  title='Anime images',
  msg='Finished downloading anime images'
)
def get_images(data):
  #with alive_bar(len(data)) as bar:
  for idx_a, anime_name in enumerate(data):
    urls = data[anime_name]
    for idx, url in enumerate(urls):
      if idx >= 400:
        break
      name_clean = re.sub(r'_+', r'_', re.sub(r'[\W\s]', r'_', anime_name))
      try:
        dowload_image(url['image'], name_clean, idx)
      except Exception as e:
        print(f'Error on download image {idx + 1} of {anime_name}')
        pass
    #bar()
    print(f'Progress: {idx_a + 1}/{len(data)} - {round((idx_a + 1)/len(data)*100, 2)}%')

def get_classes_anime(path):
  classes = set()
  for filename in os.listdir(path):
    class_name, _ = filename.split('____')
    classes.add(class_name)
  return list(classes)

def wait_for_it(driver, xpath, timeout=3):
  try:
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.XPATH, xpath))
    )
  except Exception as e:
    return None

def iter_post(driver):
  anime_data = []

  xpath_next = '//a[@class="next_page"]'
  next_button = True

  while next_button is not None:
    if len(anime_data) > 400:
      break
    ul_element = wait_for_it(driver, '//ul[@id="post-list-posts"]')
    if ul_element is None:
      next_button = wait_for_it(driver, xpath_next)
      if next_button is not None:
        next_button.click()
        time.sleep(1)
      continue
    for i, li_element in enumerate(ul_element.find_elements(By.TAG_NAME, 'li')):
      a_video = li_element.find_element(By.XPATH, './a').get_attribute('href')
      a_image = li_element.find_element(By.XPATH, './div/a/img').get_attribute('src')
      anime_data.append({
        'video': a_video,
        'image': a_image
      })
    next_button = wait_for_it(driver, xpath_next)
    if next_button is not None:
      next_button.click()
      time.sleep(rd.randint(1, 2))
  return anime_data

def get_images(url, driver, anime_name):
  url_search = url + anime_name
  driver.get(url_search)
  return iter_post(driver)

def get_names(driver):
  names = []
  xpath_next = '//a[@class="next_page"]'
  next_button = wait_for_it(driver, xpath_next)
  
  while next_button is not None:
    for tr_element in driver.find_elements(By.XPATH, '//table[@class="highlightable"]/tbody/tr'):
      try:
        amount_post = tr_element.find_element(By.XPATH, './td[1]').text
        amount_post = int(amount_post)
        if amount_post >= 10:
          a_name = tr_element.find_element(By.XPATH, './td[2]/a[2]' ).text
          names.append(a_name)
      except Exception as e:
        print(e)
        pass
    next_button.click()
    time.sleep(rd.randint(1, 2))
    next_button = wait_for_it(driver, xpath_next)
  return names

def get_score(anime_name, driver):
  url_search = f'https://myanimelist.net/anime.php?cat=anime&q={anime_name}'
  driver.get(url_search)
  score = 0
  for filename in os.listdir(path):
    class_name, _ = filename.split('____')
    score += 1
  return score

def relevant_anime(anime_name, df_anime, amount_table):
  
  if amount_table.get(anime_name, 0) < 250:
    return False

  anime_name = re.sub(r'_', r' ', anime_name)
  df_result = df_anime[df_anime['name'].str.contains(anime_name)]

  if df_result.empty:
    anime_name = ' '.join(anime_name.split(' ')[:3])
    df_result = df_anime[df_anime['name'].str.contains(anime_name)]
  return not df_result.empty

def amount_anime_table(datapath):
  dic = {}
  for filename in os.listdir(datapath):
    class_name, _ = filename.split('____')
    dic[class_name] = dic.get(class_name, 0) + 1
  return dic

#### Get anime images

In [ ]:
anime_data = json.load(open(DATASET_JSON_PATH))
get_images(anime_data)

amount = amount_anime_table(DATASET_PATH)
pkl.dump(amount, open(AMOUNT_TABLE_PATH, 'wb'))

#### Filter animes

In [ ]:
df = pd.read_pickle('./data/df_anime_rank.pkl')

amount_table = pkl.load(open(AMOUNT_TABLE_PATH, 'rb'))
all_class_array = get_classes_anime(DATASET_PATH)

class_array = set()
for anime_name in all_class_array:
  if relevant_anime(anime_name, df, amount_table):
    class_array.add(anime_name)

class_array = list(class_array)
class_array = class_array[:24]
print(f'All classes: {len(all_class_array)} - Filtered {len(class_array)}')
del all_class_array

### TF functions

In [ ]:
def get_class_id(class_name):
  return class_array.index(class_name)

def build_example(path_file, class_name):
  img_array = open(path_file, 'rb').read()
  
  #img = load_img(path_file, target_size=(SIZE_IMG, SIZE_IMG))
  #img_array = np.array(img)
  #img_array = preprocess_input(img_array, mode='tf')
  #key = hashlib.sha256(img_array).hexdigest()
  example = tf.train.Example(
    features=tf.train.Features(feature={
    #'key': tf.train.Feature(bytes_list=tf.train.BytesList(value=[key.encode('utf-8')])),
    'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_array])),
    #'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_array.tobytes()])),
    'class_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[get_class_id(class_name)])),
    #'class_name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[class_name.encode('utf-8')])),
    #'filepath': tf.train.Feature(bytes_list=tf.train.BytesList(value=[path_file.encode('utf-8')]))
  }))
  return example

def create_tfrecord(data_path, withe_list):
  files = os.listdir(data_path)
  writer = tf.io.TFRecordWriter(TFRECORD_PATH)
  
  print('Started creating tfrecord')
  for idx, filename in enumerate(files):
    class_name, _ = filename.split('____')
  
    if class_name in withe_list:
      path_file = os.path.join(data_path, filename)
      tf_example = build_example(path_file, class_name)
      writer.write(tf_example.SerializeToString())
  print('Finished creating tfrecord')
  writer.close()

def parse_tfrecord(tfrecord, size):
  x = tf.io.parse_single_example(tfrecord, IMAGE_FEATURE_MAP)
  x_train = tf.image.decode_jpeg(x['image'], channels=3)
  x_train = tf.image.resize(x_train, (size, size))

  #class_id = tf.sparse.to_dense(x['class_id'], default_value=-1)
  class_id = x['class_id']
  if class_id is None:
    class_id = -1

  labels = tf.cast(class_id, tf.int64)
  y_train = labels
  #y_train = tf.stack([ labels ], axis=1)
  return x_train, y_train

def load_tfrecord_dataset(file_pattern, size):
  files = tf.data.Dataset.list_files(file_pattern)
  dataset = files.flat_map(tf.data.TFRecordDataset)
  return dataset.map(lambda x: parse_tfrecord(x, size))

IMAGE_FEATURE_MAP = {
  'image': tf.io.FixedLenFeature([], tf.string),
  'class_id': tf.io.FixedLenFeature([], tf.int64)
}

if True:
  os.remove(TFRECORD_PATH)
  create_tfrecord(DATASET_PATH, class_array)

### Evaluate models

In [ ]:
def create_model(num_classes, input_shape, units, type_extractor = 'vgg') -> tf.keras.Model:
  if type_extractor == 'vgg':
    feature_extractor = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
  elif type_extractor == 'inception':
    feature_extractor = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
  elif type_extractor == 'resnet':
    feature_extractor = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
  else:
    raise ValueError('type_extractor must be vgg, inception or resnet')
  
  model = tf.keras.Sequential()
  #model.add(tf.keras.layers.Input(input_shape, name='input'))
  model.add(feature_extractor)
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units,activation=tf.nn.relu))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(units,activation=tf.nn.relu))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(units,activation=tf.nn.relu))
  model.add(tf.keras.layers.Dropout(0.5))
  #new
  model.add(tf.keras.layers.Dense(units,activation=tf.nn.relu))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(units,activation=tf.nn.relu))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(units,activation=tf.nn.relu))
  model.add(tf.keras.layers.Dropout(0.5))

  model.add(tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax))
  return model

def train(model, train_ds, val_ds, units, epochs=15, mode='fit', type_model='vgg'):
  logdir = "logs/scalars/" + time.strftime("%Y%m%d_%H-%M-%S")
  #logdir = "logs/scalars/" + "test_replicated_seed_5"
  if mode == 'eager_tf':
    avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)
    avg_val_loss = tf.keras.metrics.Mean('val_loss', dtype=tf.float32)
    
    for epoch in range(1, epochs + 1):
      for batch, (images, labels) in enumerate(train_ds):
        with tf.GradientTape() as tape:
          outputs = model(images, training=True)
          regularization_loss = tf.reduce_sum(model.losses)
          pred_loss = []
          for output, label, loss_fn in zip(outputs, labels, loss):
            pred_loss.append(loss_fn(label, output))
          total_loss = tf.reduce_sum(pred_loss) + regularization_loss
        grads = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        print("{}_train_{}, {}, {}".format(
          epoch, batch, total_loss.numpy(),
          list(map(lambda x: np.sum(x.numpy()), pred_loss))
        ))
        avg_loss.update_state(total_loss)
  elif mode == 'fit':
    callbacks = [
      ReduceLROnPlateau(verbose=1),
      EarlyStopping(patience=4, verbose=1),
      ModelCheckpoint(
        f'checkpoints/{type_model}_{units}_units.h5', 
        verbose=1,
        monitor='accuracy',
        save_freq='epoch',
        save_best_only=True,
        save_weights_only=False,
      ),
      TensorBoard(log_dir=logdir, histogram_freq=1)
    ]

    start_time = time.time()
    model.fit(
      train_ds,
      epochs=epochs,
      callbacks=callbacks,
      validation_data=val_ds
    )
    end_time = time.time() - start_time
    print(f'Total Training Time: {end_time} seconds')

In [ ]:
SIZE_IMG=224#224#416
tf_record = load_tfrecord_dataset(TFRECORD_PATH, SIZE_IMG)
all_ds_len = sum(1 for _ in tf_record)
print(f'Total number of images: {all_ds_len}')

#len_mini = all_ds_len
#mini_tf_record = tf_record.take(len_mini)

n_train = int(all_ds_len * 0.8)
n_valid = int(all_ds_len * 0.1)
n_test = all_ds_len - n_train - n_valid

tf_record = tf_record.shuffle(n_train + n_valid + n_test, seed=SEED)
train_ds = tf_record.take(n_train)
valid_ds = tf_record.skip(n_train).take(n_valid)
test_ds = tf_record.skip(n_train + n_valid).take(n_test)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001, clipnorm=1.0)
#0.0001
#0.0005 - 110312 - batch 8 - 1 best
#0.00075 - 111326 - batch 12 - 2 best
#prev 256, without all the new on create_model
loss = tf.keras.losses.SparseCategoricalCrossentropy()
UNITS = 256 #128 #256 #512
model = create_model(
  num_classes=len(class_array),
  input_shape=(SIZE_IMG, SIZE_IMG, 3),
  type_extractor='vgg',
  units=UNITS
)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
train(
  model=model,
  epochs=50,
  units=UNITS,
  val_ds=valid_ds.batch(32),
  train_ds=train_ds.batch(32),
  mode='fit', type_model='vgg'
)

In [31]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_5 (Flatten)         (None, 25088)             0         
                                                                 
 dense_40 (Dense)            (None, 256)               6422784   
                                                                 
 dropout_35 (Dropout)        (None, 256)               0         
                                                                 
 dense_41 (Dense)            (None, 256)               65792     
                                                                 
 dropout_36 (Dropout)        (None, 256)               0         
                                                                 
 dense_42 (Dense)            (None, 256)              

In [ ]:
model.save(f'models/{type_model}.h5')

In [ ]:
%load_ext tensorboard

In [ ]:
for x, y in tf_record:
  print(x.shape, y.shape)
  break

In [ ]:
tf_record = load_tfrecord_dataset(TFRECORD_PATH)
#get length of tf_record
len(list(tf_record))